In [40]:
from estnltk import Text
a = Text('Kui Arno isaga koolimajja jõudis, olid tunnid juba alanud.')
a.text

'Kui Arno isaga koolimajja jõudis, olid tunnid juba alanud.'

In [41]:
a.tag_layer(["morph_analysis"])

Text(text="Kui Arno isaga koolimajja jõudis, olid tunnid juba alanud.")

In [42]:
def JS_data (text, span_name):
    span = getattr(text, span_name)
    spancount = len(span.spans)
    spanstarts = []
    spanends = []
    for i in range(spancount):
        spanstart = span[i].start
        spanend = span[i].end
        spanstarts.append(spanstart)
        spanends.append(spanend)
    return text.text, spanstarts, spanends

In [43]:
t = '''
<html>
<style>
.boxes {
    background: linear-gradient(to bottom right, white, lightgray);
    border: 1px solid black;
    border-radius: 4px;
    box-shadow: 4px 4px 4px gray;
    width: fit-content;
}
</style>
<body>

<p id="maintext">'''
t+=JS_data(a, "morph_analysis")[0]
t+='''</p>

<script>

function box(content) {
    let textbox = document.createElement("DIV");
    textbox.classList.add("boxes");
    let t = document.createTextNode(content);
    textbox.appendChild(t);
    let textNode = document.getElementById("maintext");
    let previousBox = document.querySelector('.boxes');
    if (previousBox){
        previousBox.parentNode.removeChild(previousBox);
    }
    textNode.appendChild(textbox);
}

function add_spans() {
    let spanstarts = '''
t+=str(JS_data(a, "morph_analysis")[1])
t+=''';
    let spanends = '''
t+=str(JS_data(a, "morph_analysis")[2])
t+=''';
    let spancount = spanstarts.length;
    let text = document.getElementById("maintext");
    let innerHTML = text.innerHTML;
    for (let i = 0; i < spancount; i++) {
        innerHTML = innerHTML.substring(0,spanstarts[i]+i*41) + "<span onclick=box(this.innerHTML)>"+
        innerHTML.substring(spanstarts[i]+i*41,spanends[i]+i*41) + "</span>" +
        innerHTML.substring(spanends[i]+i*41,innerHTML.length);
        text.innerHTML=innerHTML;
    }
    
}

add_spans()

</script>

</body>
</html>
'''

class T:
    def _repr_html_(self):
        return t
T()